In [1]:
import glob
import pandas as pd
import json
import subprocess
import yaml
import numpy as np

In [2]:
results_path = "../benchmark-results/"

In [3]:
result_files = glob.glob(results_path + "*/*")

In [4]:
results = []

for file in result_files:
    with open(file) as f:
        data = json.load(f)
    results.append(pd.json_normalize(data))

# Merging all results here

In [5]:
df = pd.concat(results, ignore_index=True)

In [6]:
df

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.max_cpu_utilization,run_stats.max_memory_utilization,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan
0,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.953493,0.025704,0.006658,372855.07982,43908.119552,351.266126,...,0.062016,0.031250,0.023556,0.011870,9,0.046750,0.023740,0.069933,0.034966,NaN
1,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.956103,0.025953,0.004778,372855.07982,43908.119552,351.276656,...,0.062016,0.031250,0.023556,0.011870,9,0.046750,0.023740,0.069933,0.034966,NaN
2,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-1000,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.982203,0.025807,0.004750,372855.07982,43908.119552,351.381956,...,0.062016,0.031250,0.023556,0.011870,9,0.046750,0.023740,0.069933,0.034966,NaN
3,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.159104,0.025793,0.004743,372855.07982,43908.119552,35.127666,...,0.062016,0.031250,0.023572,0.011878,9,0.046781,0.023756,0.069971,0.034985,NaN
4,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.161714,0.034407,0.006400,372855.07982,43908.119552,35.138196,...,0.062016,0.031250,0.023572,0.011878,9,0.046781,0.023756,0.069971,0.034985,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1953067,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-1250-100,HEFT,True,4717.220406,0.003664,0.001350,37701.25071,68387.007305,134.640868,...,1.000000,0.022023,0.888029,0.020179,9,0.888029,0.020179,0.888793,0.020199,4717.220306
1953068,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-1250-1000,HEFT,True,4717.221306,0.004411,0.001248,37701.25071,68387.007305,135.023368,...,1.000000,0.022023,0.888029,0.020179,9,0.888029,0.020179,0.888793,0.020199,4717.220306
1953069,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-12500-10,HEFT,True,4717.220309,0.003863,0.001507,37701.25071,68387.007305,13.461408,...,1.000000,0.022023,0.888029,0.020179,9,0.888029,0.020179,0.888793,0.020199,4717.220299
1953070,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-12500-100,HEFT,True,4717.220399,0.004778,0.002132,37701.25071,68387.007305,13.499658,...,1.000000,0.022023,0.888029,0.020179,9,0.888029,0.020179,0.888793,0.020199,4717.220299


# Finding DAG file paths

In [7]:
import os
import argparse

def find_file(target_filename, search_path="."):
    """
    Recursively search for files with the name target_filename starting from search_path.
    
    Parameters:
      search_path (str): The directory to start the search (absolute or relative).
      target_filename (str): The exact name of the file to search for.
      
    Returns:
      Full file path that match the target_filename, or False if
      none was found.
    """

    for root, dirs, files in os.walk(search_path):
        for file in files:
            if target_filename in file:
                return os.path.join(root, file)
    
    print("Error: File not found {}".format(target_filename))
    sys.exit(1)


def file_finder_helper(unique_files, search_path):
    file_paths = {}

    for file_name in unique_files:
        file_path = find_file(file_name, search_path=search_path)
    
        if file_path in list(file_paths.values()):
            print("Error: Path already in dictionary. This is a duplicate: " + file_path)
            sys.exit(1)
        else:
            file_paths[file_name] = file_path

    return file_paths

In [8]:
# find_file('1000genome-chameleon-10ch-100k-001.json', os.getcwd() + "/../dags")

In [9]:
dag_paths = file_finder_helper(
    df["dag"].unique(), os.getcwd() + "/../dags"
)

In [10]:
df["dag_path"] = df["dag"].map(dag_paths)

In [11]:
# Make sure all dags were found
df[df["dag_path"] == False]

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.max_memory_utilization,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path


In [12]:
df[df["dag_path"] == ""]

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.max_memory_utilization,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path


In [13]:
df.head()["dag_path"][0]

'/home/hgsm/dslab-fork/dytas-benchmark/dytas-comparisons/analysis/../dags/daggen/150-373-0.1.yaml'

# Now finding system file paths

In [14]:
system_paths = file_finder_helper(df["system"].unique(), os.getcwd() + "/../systems")

In [15]:
df["system_path"] = df["system"].map(system_paths)

In [16]:
df[df["system_path"] == False]

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path,system_path


In [17]:
df[df["system_path"] == ""]

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path,system_path


In [18]:
df.head()["system_path"][0]

'/home/hgsm/dslab-fork/dytas-benchmark/dytas-comparisons/analysis/../systems/16-128/het/cluster-het-16-128-ConstantBandwidth-125-10.yaml'

# Loading each DAG's properties as it would be parsed by DSLab

In [19]:
path_to_dag_export = "/home/hgsm/dslab-fork/target/release/dag-export"

def get_tasks(dag_path):
    result = subprocess.run(
        [
            path_to_dag_export,
            '--dag-path',
            dag_path
        ],
        capture_output=True,
        text=True
    )

    if result.returncode != 0:
        raise RuntimeError(f"Rust executable failed: {result.stderr}")

    response_json = result.stdout.strip()
    tasks = json.loads(response_json)

    return tasks


def load_dags(dag_paths):
    dags = {}
    for path in dag_paths:
        dags[path] = get_tasks(path)

    return dags

In [20]:
cached_dags = load_dags(df.dag_path.unique())

In [21]:
# list(cached_dags.values())[0]

# Measuring the DAGs properties
#### Properties: number of nodes and edges; graph density.


$$
DEN_{Undirected|Acyclic}=\frac{2|E|}{|V|*(|V|-1)}
$$

In [22]:
def calculate_dag_properties(dag_tasks):
    """
    Calculate the number of nodes, edges and density of a DAG.
        
    Each task is considered a node. The 'parents' field in each
    task lists its incoming edges.
    """
    if not dag_tasks:
        return 0, 0, 0  # If no tasks, then nodes, edges and density are all zero.

    num_nodes = len(dag_tasks)

    # For graphs with fewer than 2 nodes, density is not defined (we return 0)
    if num_nodes < 2:
        return num_nodes, 0, 0.0
    
    # Sum up the edges from the length of the "parents" list for each task.
    num_edges = sum(len(task["parents"]) for task in dag_tasks)
    
    # Compute density using the DAG density equation
    # (Actual number of edges vs maximum number of possible edges)
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1))

    return (num_nodes, num_edges, density)

In [23]:
unique_dags = df["dag_path"].unique()

In [24]:
# unique_dags

In [25]:
dag_properties = {}
for dag_path in unique_dags:
    properties = calculate_dag_properties(cached_dags[dag_path])
    dag_properties[dag_path] = properties

In [26]:
properties_df = df["dag_path"].map(dag_properties)

In [27]:
df["dag_nodes"] = properties_df.apply(lambda x: x[0]) # node count

In [28]:
df["dag_edges"] = properties_df.apply(lambda x: x[1]) # edge count

In [29]:
df["dag_density"] = properties_df.apply(lambda x: x[2]) # density

In [30]:
df.head()

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path,system_path,dag_nodes,dag_edges,dag_density
0,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.953493,0.025704,0.006658,372855.07982,43908.119552,351.266126,...,0.046750,0.023740,0.069933,0.034966,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602
1,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.956103,0.025953,0.004778,372855.07982,43908.119552,351.276656,...,0.046750,0.023740,0.069933,0.034966,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602
2,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-1000,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.982203,0.025807,0.004750,372855.07982,43908.119552,351.381956,...,0.046750,0.023740,0.069933,0.034966,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602
3,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.159104,0.025793,0.004743,372855.07982,43908.119552,35.127666,...,0.046781,0.023756,0.069971,0.034985,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602
4,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.161714,0.034407,0.006400,372855.07982,43908.119552,35.138196,...,0.046781,0.023756,0.069971,0.034985,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602


In [31]:
df.dag_density.max()

0.4329004329004329

# Calculating Speedup
Sequential execution time to parallel execution time.

$$
\text{Speedup} = 
  \frac{\displaystyle \min_{p_j \in Q}\!\Bigl\{\sum_{n_{i} \in V} \omega_{i,j}\Bigr\}}
       {\text{makespan}}
$$

In [32]:

def calculate_min_task_execution_times(dag_tasks, system):
    """
    For each of the tasks in the dag it will find the
    execution time of the task as if it were being executed
    on the processor that will minimize its executiion time
    the most.
    """
    min_task_execution_times = {}
    
    for task in dag_tasks:
        minimum_execution_time = np.inf
        for node in system['resources']:
            # Depending on how many cores the task
            # requires, and is able
            # to be assigned, this calculation will be
            # modified, since the speed of each node
            # is per core.

            cores_used = min(node["cores"], task["max_cores"])
            execution_time = task["flops"] / (node["speed"] * cores_used)

            minimum_execution_time = min(minimum_execution_time, execution_time)

        min_task_execution_times[task["id"]] = minimum_execution_time

    return min_task_execution_times
    

# def calculate_min_sequential_execution_time(dag_tasks, system):
#     """
#     Calculates the execution time (makespan) of the workflow only
#     taking into consideration task execution times as if they
#     were being executed on the fastest processor.
#     """
    
#     min_task_execution_times = calculate_min_task_execution_times(
#         dag_tasks,
#         system
#     )

#     return sum(min_task_execution_times.values())


# def calculate_speedup(makespan, dag_tasks, system):
#     min_sequential_execution_time = calculate_min_sequential_execution_time(dag_tasks, system)
#     return min_sequential_execution_time / makespan

In [33]:
def load_systems(system_paths):
    systems = {}
    for path in system_paths:
        with open(path) as f:
            system = yaml.safe_load(f)
            
        systems[path] = system

    return systems

In [34]:
cached_systems = load_systems(df.system_path.unique())

In [35]:
# Caching each system-dag pair minimum task execution times
sys_dag_min_task_exec_times = {}

for dag_path, dag_tasks in cached_dags.items():
    for system_path, system in cached_systems.items():
        system_dag_pair = system_path + "_" + dag_path

        try:
            # If value was already calculated, continue.
            value = sys_dag_min_task_exec_times[system_dag_pair]
        except:
            # Otherwise, calculate and store the value
            sys_dag_min_task_exec_times[
                system_dag_pair
            ] = calculate_min_task_execution_times(dag_tasks, system)

In [36]:
# sys_dag_min_task_exec_times

In [37]:
df.head()

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path,system_path,dag_nodes,dag_edges,dag_density
0,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.953493,0.025704,0.006658,372855.07982,43908.119552,351.266126,...,0.046750,0.023740,0.069933,0.034966,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602
1,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.956103,0.025953,0.004778,372855.07982,43908.119552,351.276656,...,0.046750,0.023740,0.069933,0.034966,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602
2,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-1000,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.982203,0.025807,0.004750,372855.07982,43908.119552,351.381956,...,0.046750,0.023740,0.069933,0.034966,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602
3,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.159104,0.025793,0.004743,372855.07982,43908.119552,35.127666,...,0.046781,0.023756,0.069971,0.034985,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602
4,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.161714,0.034407,0.006400,372855.07982,43908.119552,35.138196,...,0.046781,0.023756,0.069971,0.034985,NaN,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602


In [38]:
df["speedup"] = df.apply(
    lambda row:
    sum(sys_dag_min_task_exec_times[
        row["system_path"] + "_" + row["dag_path"]
    ].values()) / row["makespan"],
    axis=1
)

# Calculating Efficiency

$$
\text{Efficiency} = \frac{\text{Speedup}}{\text{No. Processors in the System}}
$$

In [39]:
df["processor_count"] = df["system_path"].apply(
    lambda x: len(cached_systems[x]["resources"])
)

In [40]:
df["processor_count"]

0          16
1          16
2          16
3          16
4          16
           ..
1953067     8
1953068     8
1953069     8
1953070     8
1953071     8
Name: processor_count, Length: 1953072, dtype: int64

In [41]:
df["core_count"] = df["system_path"].apply(lambda x: sum([node["cores"] for node in cached_systems[x]["resources"]]))

In [42]:
df["core_count"]

0          128
1          128
2          128
3          128
4          128
          ... 
1953067      8
1953068      8
1953069      8
1953070      8
1953071      8
Name: core_count, Length: 1953072, dtype: int64

In [43]:
df["efficiency_processors"] = df.speedup / df.processor_count

In [44]:
df["efficiency_cores"] = df.speedup / df.core_count

In [45]:
df.head()

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,dag_path,system_path,dag_nodes,dag_edges,dag_density,speedup,processor_count,core_count,efficiency_processors,efficiency_cores
0,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.953493,0.025704,0.006658,372855.07982,43908.119552,351.266126,...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602,2.269356,16,128,0.141835,0.017729
1,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.956103,0.025953,0.004778,372855.07982,43908.119552,351.276656,...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602,2.269356,16,128,0.141835,0.017729
2,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-1000,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.982203,0.025807,0.004750,372855.07982,43908.119552,351.381956,...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602,2.269356,16,128,0.141835,0.017729
3,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.159104,0.025793,0.004743,372855.07982,43908.119552,35.127666,...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602,2.270888,16,128,0.141931,0.017741
4,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.161714,0.034407,0.006400,372855.07982,43908.119552,35.138196,...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,/home/hgsm/dslab-fork/dytas-benchmark/dytas-co...,150,152,0.013602,2.270888,16,128,0.141931,0.017741


# Obtaining system network properties

In [46]:
network_cols = [
    "network_bandwidth", 
    "network_latency", 
    "network_model"
]

df[network_cols] = df["system_path"].apply(
    lambda x: pd.Series([
        cached_systems[x]["network"]["bandwidth"],
        cached_systems[x]["network"]["latency"],
        cached_systems[x]["network"]["model"]
    ]
    )
)

In [47]:
df[network_cols]

,network_bandwidth,network_latency,network_model
0,125,10,ConstantBandwidth
1,125,100,ConstantBandwidth
2,125,1000,ConstantBandwidth
3,1250,10,ConstantBandwidth
4,1250,100,ConstantBandwidth
...,...,...,...
1953067,1250,100,SharedBandwidth
1953068,1250,1000,SharedBandwidth
1953069,12500,10,SharedBandwidth
1953070,12500,100,SharedBandwidth


# Now computing the SLR
#### Schedule Length Ratio (SLR): A normalized makespan

$$
\mathrm{SLR} = \frac{\mathrm{makespan}}{\sum_{n_i \in CP_{MIN}} \min_{p_j \in Q} \{\omega_{i,j}\}}.
$$

In [48]:
from collections import deque

def find_critical_path(dag_tasks):
    """
    Will find the critical path of the DAG.
    """

    dp = {}
    # Initializing dp for nodes without parents (start nodes).
    # Each value will contain a tuple of:
    # (max total time to this node, predecessor node id)
    for i, task in enumerate(dag_tasks):
        if not task["parents"]:
            dp[i] = (task['min_exec_time'], None)

    # Contains how many parents have been satisfied per node
    indegree = {i : len(task["parents"]) for i,task in enumerate(dag_tasks)}

    # Contains nodes without parents (start nodes)
    queue = deque([i for i, task in enumerate(dag_tasks) if indegree[i] == 0])

    while queue:
        current = queue.popleft()
        current_time = dp[current][0]
        for child in dag_tasks[current]['children']:
            new_time = current_time + dag_tasks[child]["min_exec_time"]

            # We update if this new time is longer than prior one
            if child not in dp or new_time > dp[child][0]:
                dp[child] = (new_time, current)
            indegree[child] -= 1
            if indegree[child] == 0:
                queue.append(child)

    # Task with the max accumulated time (critical path time).
    max_task = max(dp, key=lambda i: dp[i][0])
    max_time = dp[max_task][0]

    # Reconstructing the critical path by backtracking
    critical_path = []
    task = max_task
    while task is not None:
        critical_path.append(task)
        task = dp[task][1]

    critical_path.reverse()
    
    return (critical_path, max_time)

In [49]:
# Caching the critical path for each system dag pair
# (each DAG's critical path will be different
# according to the system it is being executed in)
sys_dag_critical_path = {}
# import time
for dag_path, dag_tasks in cached_dags.items():
    for system_path, system in cached_systems.items():
        system_dag_pair = system_path + "_" + dag_path

        min_task_execution_times = sys_dag_min_task_exec_times[system_dag_pair]
        
        copy_dag_tasks = dag_tasks.copy()
        
            # Map the min execution time to each task in the DAG.
        for i, exec_time in min_task_execution_times.items():
            copy_dag_tasks[i]["min_exec_time"] = exec_time
        
        try:
            # If value was already calculated, continue.
            value = sys_dag_critical_path[system_dag_pair]
        except:
            # Otherwise, calculate and store the value
            sys_dag_critical_path[
                system_dag_pair
            ] = find_critical_path(copy_dag_tasks)

In [50]:
# sys_dag_critical_path

In [51]:
df["SLR"] = df.apply(
    lambda row: 
        row["makespan"] / sys_dag_critical_path[row["system_path"] + "_" + row["dag_path"]][1],
        axis=1
)

In [52]:
df.head()

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,dag_density,speedup,processor_count,core_count,efficiency_processors,efficiency_cores,network_bandwidth,network_latency,network_model,SLR
0,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.953493,0.025704,0.006658,372855.07982,43908.119552,351.266126,...,0.013602,2.269356,16,128,0.141835,0.017729,125,10,ConstantBandwidth,3.255511
1,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.956103,0.025953,0.004778,372855.07982,43908.119552,351.276656,...,0.013602,2.269356,16,128,0.141835,0.017729,125,100,ConstantBandwidth,3.255511
2,150-373-0.1,cluster-het-16-128-ConstantBandwidth-125-1000,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122700.982203,0.025807,0.004750,372855.07982,43908.119552,351.381956,...,0.013602,2.269356,16,128,0.141835,0.017729,125,1000,ConstantBandwidth,3.255511
3,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.159104,0.025793,0.004743,372855.07982,43908.119552,35.127666,...,0.013602,2.270888,16,128,0.141931,0.017741,1250,10,ConstantBandwidth,3.253314
4,150-373-0.1,cluster-het-16-128-ConstantBandwidth-1250-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,122618.161714,0.034407,0.006400,372855.07982,43908.119552,35.138196,...,0.013602,2.270888,16,128,0.141931,0.017741,1250,100,ConstantBandwidth,3.253314


In [53]:
# df.head()["dag_path"][0]

In [54]:
# calculate_schedule_length_ratio(df.head()["makespan"][0],
#     cached_dags[df.head()["dag_path"][0]],
#     cached_systems[df.head()["system_path"][0]]
# )

In [55]:
# df["SLR"] = df.apply(
#     lambda row: pd.Series(calculate_schedule_length_ratio(
#         row["makespan"],
#         cached_dags[row["dag_path"]],
#         row["system_path"] + "_" + row["dag_path"]
#     )),
#     axis=1
# )

# Applying workflow names

In [56]:
workflows = [
    "blast", "bwa", "1000genome",
    "cycles", "epigenomics", "montage",
    "seismology", "soykb", "srasearch"
]

In [57]:
for workflow in workflows:
    df.loc[df.dag.str.contains(workflow), "workflow"] = workflow

In [58]:
# Assigning "random" to the onees generated
# with daggen
df.loc[df.workflow.isna(), "workflow"] = "random"

In [59]:
df.workflow

0             random
1             random
2             random
3             random
4             random
             ...    
1953067    srasearch
1953068    srasearch
1953069    srasearch
1953070    srasearch
1953071    srasearch
Name: workflow, Length: 1953072, dtype: object

# Calculating the Communication-to-Computation Ratio (CCR)

$$
\text{CCR} = \frac{\sum_{e \in E} c(e)}{\sum_{v \in V} w(v)}
$$

Where the numerator is the total data transfer time and the denominator is the total task computation time, for the workflow or DAG.

In [60]:
df["CCR"] = df["run_stats.total_network_time"] / df["run_stats.total_task_time"]

# Creating a test id

Each (system, dag) pair with be a test instance.

In [61]:
df["test_id"] = df["system"].astype(str) + "_" + df["dag"].astype(str)

In [62]:
base_filename = "pre-processed-metrics-dataframe"

df.to_parquet(
    "./data/{}.parquet".format(base_filename),
    engine="pyarrow",
    compression="gzip"
)

df.to_pickle("./data/{}.pkl".format(base_filename))

df.to_csv("./data/{}.csv".format(base_filename), index=False)